In [53]:
from bs4 import BeautifulSoup, Comment
import pandas as pd
import os
import time
import requests

Lets get 2021-2022 CL stats per game per team. We need per game stats. 

In [2]:
league_url = 'https://fbref.com/en/comps/8/2021-2022/2021-2022-Champions-League-Stats'

In [3]:
data = requests.get(league_url)    # getting the leage table for 2021-2022 season

Extracting squad urls orm the league table

In [14]:
soup = BeautifulSoup(data.text)
league_table = soup.select('#results2021-202280_overall')[0]
links = league_table.find_all('a')
links = [l.get('href') for l in links] # extract hrefs for a tag

This gives us all the links on that table, including the best scorers etc. But we only want the squad urls, so we only select links that have the season in their url, as they are for the teams. 

In [15]:
links = [l for l in links if "2021-2022" in l]

### If we were only interested in scraping the champions league stats.

Construct an actual url to the team's page, and only get the champions league data!
```len('/2021-2022/')``` = 11 and index itself is 19 in this case you may need to change the year numbers and make them dynamic later on for more seasons. 
Below we process the links we have to get tot he scores and fixtures table for each team and for the Champions league only as we only use the games the team has played in the Champions League. 

``` index = links[0].find('/2021-2022/')```
```links = [l[:index+11] + 'matchlogs/c8/schedule/' +l[11+index:]+'-Champions-League' for l in links]```
```team_urls = [f"https://fbref.com/{l}" for l in links]```

### However:

The issue is that in that case we do not have that many games to train our data, and we would need more games. 
What we can do however, is to get all of he data for all the competitions, say Real Madrid will ave far more La Liga games, or Liverpool will have many more Premier League games than Champions League games. Fortunately, we have all the data for the opponents too, so we can have a much more populated dataset. And since the club that takes part in country-wide agues is the same as the one taking part in the Champions league, we can have a much better understanding of their performance.

In [16]:
team_urls = [f"https://fbref.com/{l}" for l in links]

Later on we loop and go through every team, for now, lets select the first team on the list.

In [22]:
data = requests.get(team_urls[0])
games = pd.read_html(data.text, match = 'Scores & Fixtures')

In [24]:
games[0][0:5] # Real Madrid's games in the 2021 to 2022 season.

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2021-08-14,22:00,La Liga,Matchweek 1,Sat,Away,W,4,1,Alavés,1.7,1.7,58,3968.0,Karim Benzema,4-3-3,César Soto,Match Report,NaN
1,2021-08-22,22:00,La Liga,Matchweek 2,Sun,Away,D,3,3,Levante,1.6,1.9,61,9838.0,Karim Benzema,4-3-3,Guillermo Cuadra,Match Report,NaN
2,2021-08-28,22:00,La Liga,Matchweek 3,Sat,Away,W,1,0,Betis,0.9,0.6,50,22590.0,Karim Benzema,4-3-3,Alejandro Hernández,Match Report,NaN
3,2021-09-12,21:00,La Liga,Matchweek 4,Sun,Home,W,5,2,Celta Vigo,3.2,1.8,60,19874.0,Karim Benzema,4-4-2,José Sánchez,Match Report,NaN
4,2021-09-15,21:00,Champions Lg,Group stage,Wed,Away,W,1,0,it Inter,1.2,2.4,53,37082.0,Karim Benzema,4-3-3,Daniel Siebert,Match Report,NaN


# getting the shooting stats

In [30]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]


links = [l for l in links if l and 'matchlogs/all_comps/' in l] # not all hrefs have links

links

['/en/squads/53a2f082/2021-2022/matchlogs/all_comps/schedule/Real-Madrid-Scores-and-Fixtures-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/shooting/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/keeper/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/passing/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/passing_types/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/gca/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/defense/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/possession/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/misc/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2021-2022/matchlogs/all_comps/schedule/Real-Madr

In [31]:
stats_urls = [f"https://fbref.com/{l}" for l in links]

In [119]:
stats_urls[1:9][1] # get from 1 to 9 to get all these tables [0] is for the shooting stats.

'https://fbref.com//en/squads/53a2f082/2021-2022/matchlogs/all_comps/keeper/Real-Madrid-Match-Logs-All-Competitions'

In [116]:

stats = pd.read_html(stats_urls[1:9][0], attrs={'id': 'matchlogs_for'})[0]
stats.head()

For Real Madrid                                                            \
             Date   Time          Comp        Round  Day Venue Result GF GA   
0      2021-08-14  22:00       La Liga  Matchweek 1  Sat  Away      W  4  1   
1      2021-08-22  22:00       La Liga  Matchweek 2  Sun  Away      D  3  3   
2      2021-08-28  22:00       La Liga  Matchweek 3  Sat  Away      W  1  0   
3      2021-09-12  21:00       La Liga  Matchweek 4  Sun  Home      W  5  2   
4      2021-09-15  21:00  Champions Lg  Group stage  Wed  Away      W  1  0   

               ... Standard               Expected                            \
     Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG np:G-xG   
0      Alavés  ...     18.7  2.0  0     0      1.7  1.7    0.10  2.3     2.3   
1     Levante  ...     20.4  1.0  0     0      1.6  1.6    0.08  1.4     1.4   
2       Betis  ...     18.3  0.0  0     0      0.9  0.9    0.07  0.1     0.1   
3  Celta Vigo  ...     12.8  0.0  1     1      3.2  2.4    0.19  1.8     1.6   
4    it Inter  ...     16.3  0.0  0     0      1.2  1.2    0.10 -0.2    -0.2   

  Unnamed: 25_level_0  
         Match Report  
0        Match Report  
1        Match Report  
2        Match Report  
3        Match Report  
4        Match Report  

[5 rows x 26 columns]

In [117]:
stats_opp = pd.read_html(stats_urls[1:9][0], attrs={'id': 'matchlogs_against'})[0]
stats_opp.head()

Against Real Madrid                                                         \
                 Date   Time          Comp        Round  Day Venue Result GF   
0          2021-08-14  22:00       La Liga  Matchweek 1  Sat  Home      L  1   
1          2021-08-22  22:00       La Liga  Matchweek 2  Sun  Home      D  3   
2          2021-08-28  22:00       La Liga  Matchweek 3  Sat  Home      L  0   
3          2021-09-12  21:00       La Liga  Matchweek 4  Sun  Away      L  2   
4          2021-09-15  21:00  Champions Lg  Group stage  Wed  Home      L  0   

                  ... Standard               Expected                    \
  GA    Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  4      Alavés  ...     19.3  1.0  1     1      1.7  0.9    0.09 -0.7   
1  3     Levante  ...     15.1  0.0  0     0      1.9  1.9    0.23  1.1   
2  1       Betis  ...     22.1  1.0  0     0      0.6  0.6    0.05 -0.6   
3  5  Celta Vigo  ...     15.0  0.0  0     0      1.8  1.8    0.19  0.2   
4  1    it Inter  ...     14.5  1.0  0     0      2.4  2.4    0.14 -2.4   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -0.9        Match Report  
1     1.1        Match Report  
2    -0.6        Match Report  
3     0.2        Match Report  
4    -2.4        Match Report  

[5 rows x 26 columns]

In [128]:
stats = pd.read_html(stats_urls[1:9][1], attrs={'id': 'matchlogs_for'})[0]
stats.columns = stats.columns.droplevel()
stats.columns = stats.columns.str.lower()
stats.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,avglen,att,launch%,avglen,opp,stp,stp%,#opa,avgdist,match report
0,2021-08-14,22:00,La Liga,Matchweek 1,Sat,Away,W,4,1,Alavés,...,30.2,4.0,25.0,40.0,13.0,0.0,0.0,3.0,21.3,Match Report
1,2021-08-22,22:00,La Liga,Matchweek 2,Sun,Away,D,3,3,Levante,...,27.3,3.0,0.0,21.7,13.0,0.0,0.0,0.0,17.0,Match Report
2,2021-08-28,22:00,La Liga,Matchweek 3,Sat,Away,W,1,0,Betis,...,34.9,4.0,25.0,23.0,10.0,0.0,0.0,2.0,23.0,Match Report
3,2021-09-12,21:00,La Liga,Matchweek 4,Sun,Home,W,5,2,Celta Vigo,...,31.8,5.0,80.0,66.0,9.0,2.0,22.2,1.0,11.3,Match Report
4,2021-09-15,21:00,Champions Lg,Group stage,Wed,Away,W,1,0,it Inter,...,25.7,5.0,20.0,32.4,17.0,1.0,5.9,1.0,15.5,Match Report


In [130]:
stats_opp = pd.read_html(stats_urls[1:9][1], attrs={'id': 'matchlogs_against'})[0]
stats_opp.columns = stats_opp.columns.droplevel()
stats_opp.columns = stats_opp.columns.str.lower()
stats_opp.columns += "_opp"
stats_opp.head()

,date_opp,time_opp,comp_opp,round_opp,day_opp,venue_opp,result_opp,gf_opp,ga_opp,opponent_opp,...,avglen_opp,att_opp,launch%_opp,avglen_opp,opp_opp,stp_opp,stp%_opp,#opa_opp,avgdist_opp,match report_opp
0,2021-08-14,22:00,La Liga,Matchweek 1,Sat,Home,L,1,4,Alavés,...,31.9,13.0,53.8,38.3,9.0,0.0,0.0,0.0,NaN,Match Report
1,2021-08-22,22:00,La Liga,Matchweek 2,Sun,Home,D,3,3,Levante,...,43.2,11.0,45.5,32.9,19.0,0.0,0.0,1.0,19.3,Match Report
2,2021-08-28,22:00,La Liga,Matchweek 3,Sat,Home,L,0,1,Betis,...,31.6,8.0,25.0,31.3,13.0,0.0,0.0,1.0,18.5,Match Report
3,2021-09-12,21:00,La Liga,Matchweek 4,Sun,Away,L,2,5,Celta Vigo,...,42.3,2.0,100.0,56.5,12.0,0.0,0.0,3.0,22.0,Match Report
4,2021-09-15,21:00,Champions Lg,Group stage,Wed,Home,L,0,1,it Inter,...,19.9,1.0,0.0,11.0,12.0,0.0,0.0,0.0,4.0,Match Report


In [131]:
merged_stats = stats.merge(stats_opp, left_on='date', right_on='date_opp')

In [132]:
merged_stats

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,avglen_opp,att_opp,launch%_opp,avglen_opp,opp_opp,stp_opp,stp%_opp,#opa_opp,avgdist_opp,match report_opp
0,2021-08-14,22:00,La Liga,Matchweek 1,Sat,Away,W,4,1,Alavés,...,31.9,13.0,53.8,38.3,9.0,0.0,0.0,0.0,NaN,Match Report
1,2021-08-22,22:00,La Liga,Matchweek 2,Sun,Away,D,3,3,Levante,...,43.2,11.0,45.5,32.9,19.0,0.0,0.0,1.0,19.3,Match Report
2,2021-08-28,22:00,La Liga,Matchweek 3,Sat,Away,W,1,0,Betis,...,31.6,8.0,25.0,31.3,13.0,0.0,0.0,1.0,18.5,Match Report
3,2021-09-12,21:00,La Liga,Matchweek 4,Sun,Home,W,5,2,Celta Vigo,...,42.3,2.0,100.0,56.5,12.0,0.0,0.0,3.0,22.0,Match Report
4,2021-09-15,21:00,Champions Lg,Group stage,Wed,Away,W,1,0,it Inter,...,19.9,1.0,0.0,11.0,12.0,0.0,0.0,0.0,4.0,Match Report
5,2021-09-19,21:00,La Liga,Matchweek 5,Sun,Away,W,2,1,Valencia,...,44.5,9.0,66.7,41.1,11.0,0.0,0.0,0.0,10.3,Match Report
6,2021-09-22,22:00,La Liga,Matchweek 6,Wed,Home,W,6,1,Mallorca,...,49.7,8.0,50.0,37.6,11.0,0.0,0.0,0.0,1.5,Match Report
7,2021-09-25,21:00,La Liga,Matchweek 7,Sat,Home,D,0,0,Villarreal,...,29.0,9.0,44.4,42.3,24.0,2.0,8.3,2.0,12.0,Match Report
8,2021-09-28,21:00,Champions Lg,Group stage,Tue,Home,L,1,2,md Sheriff Tiraspol,...,44.5,16.0,50.0,39.1,32.0,1.0,3.1,0.0,6.0,Match Report
9,2021-10-03,16:15,La Liga,Matchweek 8,Sun,Away,L,1,2,Espanyol,...,36.2,14.0,85.7,52.6,24.0,0.0,0.0,1.0,41.0,Match Report
